In [ ]:
!python --version

In [ ]:
import tensorflow as tf
#import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# import du dataset
import pandas as pd
import seaborn as sns
import numpy as np


In [ ]:
df = pd.read_csv('../../dataset/order_classification/edible_mushrooms.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(18, 8))
ax = sns.countplot(data = df, x = 'order', alpha = 0.7)

plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
ax.set_xlabel('order')
ax.set_ylabel('nombre de photos')
ax.set_title('nombre de photos par species pour les champignons comestibles');

In [ ]:
max_size = df['order'].value_counts().max()
max_size = min(max_size, 5000)

In [ ]:
for index, value in df['order'].value_counts().items():
    print(f"{index}: {value}")

# Generating and balancing data

In [ ]:
path='../../dataset/class_order'
train_path='../../dataset/order_classification/train'
test_path='../../dataset/order_classification/test'
validation_path='../../dataset/order_classification/validation' 

In [ ]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
train_generator = train_data_generator.flow_from_directory(
    directory = train_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 4)

In [ ]:
test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
test_generator = test_data_generator.flow_from_directory(
    directory = test_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
validation_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
validation_generator = validation_data_generator.flow_from_directory(
    directory = validation_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

# Resnet50_v2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

n_class= 8#df['order'].unique().shape[0]

base_model = ResNet50V2(weights = 'imagenet', include_top = False, input_shape=(224,224,3))

for layer in base_model.layers:
    layer.trainable = False

# Get the output of the base model
base_output = base_model.output

# Add your custom layers
x = GlobalAveragePooling2D()(base_output)  # Flatten layer
x = Dense(1024, activation='relu')(x)  # Dense layer
x = Dropout(rate=0.1)(x)
x = Dense(n_class, activation='softmax')(x)  # Dense layer

# Create a new model that includes the base model and your custom layers
model_resnet = Model(inputs=base_model.input, outputs=x)


In [ ]:
model_resnet.compile(optimizer='adam', 
               loss='sparse_categorical_crossentropy', 
               metrics=['acc'])
model_resnet.summary()

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"  # specify which GPU(s) to be used

# Create an early stopping callback
early_stopping = EarlyStopping(monitor='acc', patience=5, min_delta = 0.001)

model_resnet_history = model_resnet.fit(
    train_generator,
    epochs = 20,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

In [ ]:
# Save the weights
model_resnet.save('./model_resnet_checkpoint')

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_resnet_history.history['acc'])
plt.plot(model_resnet_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_mobile_history.history['acc'])
plt.plot(model_mobile_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Confusion Matrix on test dataset

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_proba = model_mobile.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)

y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
sns.heatmap(cm, annot=True)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

## Confusion Matrix on test dataset

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_proba = model_mobile.predict(validation_generator)
y_pred = np.argmax(y_pred_proba, axis=1)

y_true = validation_generator.classes

cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
sns.heatmap(cm, annot=True)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# we can observe the tendency to classify any mushroom as part of the dominant class, a consequence of imbalancing